In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Nội dung

* [<font size=4>EDA</font>](#1)
    * [Chuẩn bị dữ liệu, lập biểu đồ](#1.1)
    * [Một số ảnh ví dụ từ tập dữ liệu](#1.2)
    * [RGB Analysis](#1.3)
    * [Parallel categories plot](#1.3)


In [ ]:
import glob
import os.path as osp
import copy

from tqdm import tqdm
tqdm.pandas()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader, Dataset

from torchvision import datasets, models
from torchvision.utils import make_grid

import os
import time
from PIL import Image
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

## Image Augmentation 

# skimage
from skimage.io import imshow, imread, imsave
from skimage.transform import rotate, AffineTransform, warp,rescale, resize, downscale_local_mean
from skimage import color,data
from skimage.exposure import adjust_gamma
from skimage.util import random_noise


# 3D scatter plot
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors


#OpenCV-Python
import cv2

# imgaug
import imageio
import imgaug as ia
import imgaug.augmenters as iaa

SAMPLE_LEN=100

# 1.  Thiết lập giá trị của các tham số cố định
Thiết lập giá trị của các tham số cố định có trong bài:
1. *num_classes*: tổng số lượng nhãn.
2. *img_size*: kích thước của ảnh sau quá trình resized bởi DataLoader.
3. *batch_size*: kích thước mỗi batch.
4. *device*: accelerator được sử dụng.
5. *criterion*: hàm mất mát (loss function) được sử dụng.

In [ ]:
class Config:
    num_classes = 12
    img_size = 224
    batch_size = 32
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    min_lr = 10**-12
    max_lr = 10
    pretrained = False
    criterion = nn.CrossEntropyLoss()
    epochs = 20

# 2. Chuẩn bị dữ liệu

In [ ]:
train_image_path = '../input/plant-pathology-2021-fgvc8/train_images'
test_image_path = '../input/plant-pathology-2021-fgvc8/test_images'
train_df_path = '../input/plant-pathology-2021-fgvc8/train.csv'
test_df_path = '../input/plant-pathology-2021-fgvc8/sample_submission.csv'

In [ ]:
df_train = pd.read_csv("/kaggle/input/plant-pathology-2021-fgvc8/train.csv")
df_sub = pd.read_csv("/kaggle/input/plant-pathology-2021-fgvc8/sample_submission.csv")

In [ ]:
# #The number of labels
# len(df_train.labels.unique())

There are 12 different labels.

In [ ]:
# #The no.values per label
# df_train.labels.value_counts()

In [ ]:
# plt.figure(figsize=(15,12))
# labels = sns.barplot(df_train.labels.value_counts().index,df_train.labels.value_counts())
# for item in labels.get_xticklabels():
#     item.set_rotation(45)

In [ ]:
# source = df_train['labels'].value_counts()

In [ ]:
# fig = go.Figure(data=[go.Pie(labels=source.index,values=source.values)])
# fig.update_layout(title='Label distribution')
# fig.show()

## Kết Luận

- Tập dữ liệu khá không cân bằng theo biểu đồ hình tròn ở trên
- Chúng tôi sẽ chọn chiến lược lấy mẫu thích hợp để giải quyết vấn đề này. Data augmentation được sử dụng để thêm các mẫu bổ sung từ các lớp thiểu số. Trong  hình ảnh của chúng tôi, điều này sẽ được xử lý được bằng cách thêm độ méo vào dữ liệu bằng cách thực hiện dịch, xoay, thay đổi tỷ lệ cũng như bằng cách thêm các loại nhiễu (áp dụng albumentation)

# Một số ảnh ví dụ từ tập dữ liệu
Chúng tôi sẽ kiểm tra kích thước của 300 hình ảnh đầu tiên

Như có thể thấy bên dưới thì tất cả các hình ảnh có kích thước khác nhau.

In [ ]:
# img_shapes = {}
# for image_name in tqdm(os.listdir(train_image_path)[:300]):
#     image = cv2.imread(os.path.join(train_image_path, image_name))
#     img_shapes[image.shape] = img_shapes.get(image.shape, 0) + 1

# print(img_shapes)

## Kết Luận
Chúng tôi thấy rằng ảnh có độ phân giải rất cao nên chúng tôi sau đó đã áp dụng resize lại ảnh để cải thiện thời gian chạy.

In [ ]:
# def visualize_batch(path,image_ids, labels):
#     plt.figure(figsize=(16, 12))
    
#     for ind, (image_id, label) in enumerate(zip(image_ids, labels)):
#         plt.subplot(3, 3, ind + 1)
#         image = cv2.imread(os.path.join(path, image_id))
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         plt.imshow(image)
#         plt.title(f"Class: {label}", fontsize=12)
#         plt.axis("off")
#     plt.show()

In [ ]:
# tmp_df = df_train.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values
# visualize_batch(train_image_path,image_ids,labels)

## 2.1 Mã hóa nhãn
Mã hóa các nhãn có trong tập dữ liệu về dạng *integer* để mô hình có thể hiểu được.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [ ]:
def encode_label(df):
    df['encoded_label'] = le.fit_transform(df.labels.values)
    return df

encode_label(df_train)
    
# Lưu từ điển mã hóa
df_labels_idx = df_train.loc[df_train.duplicated(["labels", "encoded_label"])==False]\
                [["encoded_label", "labels"]].set_index("encoded_label").sort_index()
display(df_labels_idx)

In [ ]:
# #label = complex
# tmp_df = df_train[df_train["encoded_label"] == 0]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

In [ ]:
# #label = frog_eye_leaf_spot
# tmp_df = df_train[df_train["encoded_label"] == 1]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

In [ ]:
# #label = frog_eye_leaf_spot complex
# tmp_df = df_train[df_train["encoded_label"] == 2]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

In [ ]:
# #label = healthy
# tmp_df = df_train[df_train["encoded_label"] == 3]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

In [ ]:
# #label = powdery_mildew
# tmp_df = df_train[df_train["encoded_label"] == 4]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

In [ ]:
# #label = powdery_mildew complex
# tmp_df = df_train[df_train["encoded_label"] == 5]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

In [ ]:
# #label = rust
# tmp_df = df_train[df_train["encoded_label"] == 6]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

In [ ]:
# #label = rust complex
# tmp_df = df_train[df_train["encoded_label"] == 7]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

In [ ]:
# #label = rust frog_eye_leaf_spot
# tmp_df = df_train[df_train["encoded_label"] == 8]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

In [ ]:
# #label = scab
# tmp_df = df_train[df_train["encoded_label"] == 9]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

In [ ]:
# #label = scab frog_eye_leaf_spot
# tmp_df = df_train[df_train["encoded_label"] == 10]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

In [ ]:
# #label = scab frog_eye_leaf_spot complex
# tmp_df = df_train[df_train["encoded_label"] == 11]
# print(f"Total train images for class 0: {tmp_df.shape[0]}")

# tmp_df = tmp_df.sample(9)
# image_ids = tmp_df["image"].values
# labels = tmp_df["labels"].values

# visualize_batch(train_image_path, image_ids, labels)

Chúng tôi đã plot một vài hình ảnh trong training data ở trên (các giá trị RGB có thể được nhìn thấy bằng cách di chuột qua hình ảnh). Các phần màu xanh lá cây của hình ảnh có giá trị màu xanh lam rất thấp, nhưng ngược lại, các phần màu nâu có giá trị màu xanh lam cao. Điều này cho thấy rằng các phần màu xanh lá cây (healthy) của hình ảnh có giá trị màu xanh lam thấp, trong khi các phần unhealthy có nhiều khả năng có giá trị màu xanh lam cao. **Điều này có thể cho thấy rằng kênh màu xanh lam có thể là chìa khóa để phát hiện bệnh trên cây trồng**

In [ ]:
# def load_image(image_id):
#     file_path = image_id
#     image = cv2.imread(train_image_path+'/'+ file_path)
#     return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# # Lấy 100 ảnh làm sample với SAMPLE_LEN=100 cho RBG Channel Analysis

# train_images = df_train["image"][:SAMPLE_LEN].apply(load_image)

In [ ]:
# red_values = [np.mean(train_images[idx][:, :, 0]) for idx in range(len(train_images))]
# green_values = [np.mean(train_images[idx][:, :, 1]) for idx in range(len(train_images))]
# blue_values = [np.mean(train_images[idx][:, :, 2]) for idx in range(len(train_images))]
# values = [np.mean(train_images[idx]) for idx in range(len(train_images))]

# RGB Analysis
Histogram là một biểu diễn đồ họa cho biết tần suất xuất hiện của các giá trị màu khác nhau trong hình ảnh. Trong không gian màu RGB, các giá trị pixel nằm trong khoảng từ 0 đến 255 trong đó 0 là màu đen và 255 là màu trắng. Phân tích biểu đồ có thể giúp chúng ta hiểu được phân bố độ sáng, độ tương phản và cường độ của hình ảnh. Bây giờ chúng ta hãy xem biểu đồ của một mẫu được chọn ngẫu nhiên từ mỗi danh mục.

## Phân phối Kênh Đỏ

In [ ]:
# fig = ff.create_distplot([red_values], group_labels=["R"], colors=["red"])
# fig.update_layout(showlegend=False, template="simple_white")
# fig.update_layout(title_text="Phân phối Kênh Đỏ")
# fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
# fig.data[0].marker.line.width = 0.5
# fig

### Quan sát :
Các giá trị kênh màu đỏ có vẻ gần như phân phối chuẩn, nhưng hơi lệch về bên trái (Độ lệch âm). Điều này cho thấy rằng kênh màu đỏ có xu hướng tập trung nhiều hơn ở các giá trị cao hơn, vào khoảng 100. Có sự thay đổi lớn về giá trị màu đỏ trung bình trên các hình ảnh.

## Phân phối Kênh Xanh Lá

In [ ]:
# fig = ff.create_distplot([green_values], group_labels=["G"], colors=["green"])
# fig.update_layout(showlegend=False, template="simple_white")
# fig.update_layout(title_text="Phân phối Kênh Xanh Lá")
# fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
# fig.data[0].marker.line.width = 0.5
# fig

### Quan sát:
Giá trị kênh màu xanh lá cây có phân phối đồng đều hơn giá trị kênh màu đỏ nhưng lệch phải, với đỉnh nhỏ hơn. Sự phân bố cũng có độ lệch bên phải (trái ngược với màu đỏ) và chế độ lớn hơn khoảng 160. Điều này cho thấy rằng màu xanh lá cây rõ nét hơn trong những hình ảnh này so với màu đỏ, điều này có ý nghĩa, bởi vì đây là hình ảnh của những chiếc lá!

## Phân phối Kênh Xanh Lam

In [ ]:
# fig = ff.create_distplot([blue_values], group_labels=["B"], colors=["blue"])
# fig.update_layout(showlegend=False, template="simple_white")
# fig.update_layout(title_text="Phân phối Kênh Xanh Lam")
# fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
# fig.data[0].marker.line.width = 0.5
# fig

### Quan sát:
Kênh màu xanh lam có sự phân bố đồng đều nhất trong số ba kênh màu, với độ lệch tối thiểu (lệch một chút sang trái). Kênh màu xanh lam cho thấy sự thay đổi lớn giữa các hình ảnh trong tập dữ liệu.

## Tổng hợp các kênh

In [ ]:
# fig = go.Figure()

# for idx, values in enumerate([red_values, green_values, blue_values]):
#     if idx == 0:
#         color = "Red"
#     if idx == 1:
#         color = "Green"
#     if idx == 2:
#         color = "Blue"
#     fig.add_trace(go.Box(x=[color]*len(values), y=values, name=color, marker=dict(color=color.lower())))
    
# fig.update_layout(yaxis_title="Mean value", xaxis_title="Color channel",
#                   title="Mean value vs. Color channel", template="plotly_white")

In [ ]:
# fig = ff.create_distplot([red_values, green_values, blue_values],
#                          group_labels=["R", "G", "B"],
#                          colors=["red", "green", "blue"])
# fig.update_layout(title_text="Distribution of red channel values", template="simple_white")
# fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
# fig.data[0].marker.line.width = 0.5
# fig.data[1].marker.line.color = 'rgb(0, 0, 0)'
# fig.data[1].marker.line.width = 0.5
# fig.data[2].marker.line.color = 'rgb(0, 0, 0)'
# fig.data[2].marker.line.width = 0.5
# fig

### Kết luận
Phân tích kênh màu thì thấy kênh màu xanh chủ yếu có phân bố ở vùng có intensity mạnh ( vì nó tập trung ở phần cuối đồ thị) ==> Dễ hiểu vì ảnh toàn lá. Ví dụ như ảnh toàn lá mà thấy kênh màu đỏ lại mạnh hơn thì sẽ phát hiện bất thường. Ở đây kênh màu đỏ lại phân bố ở vùng giữa mạnh hơn kênh màu xanh dương, chứng tỏ màu đỏ nó cũng có xuất hiện nhiều là do một số lá cây bị sâu bênh hay có màu đỏ

## 2.2 Tạo đường dẫn cho ảnh và phân chia tập huấn luyện, tập thẩm định

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def make_datapath_list(phase='train', val_size=0.3):
    if phase in ["train", "val"]:
        phase_path = "train_images"
    elif phase in ["test"]:
        phase_path = "test_images"
    else:
        print(f"{phase} not in path")    
        
    """
    Use resized training dataset for betting training speed
    Resized datase from: https://www.kaggle.com/ankursingh12/resized-plant2021
    """
    if phase == 'train' or phase == 'val': 
        rootpath = "/kaggle/input/resized-plant2021/img_sz_512/"
    else:
        rootpath = "/kaggle/input/plant-pathology-2021-fgvc8/test_images/"
    
    target_path = osp.join(rootpath+"/*.jpg")
    path_list = []
    
    for path in glob.glob(target_path):
        path_list.append(path)
        
    if phase in ["train", "val"]:
        train, val = train_test_split(path_list, test_size=val_size, random_state=47, shuffle=True)
        if phase == "train":
            path_list = train
        else:
            path_list = val
    
    return path_list

In [ ]:
train_list = make_datapath_list(phase='train')
print(f'The length of training set: {len(train_list)}')
val_list = make_datapath_list(phase='val')
print(f'The length of valuation set: {len(val_list)}')
test_list = make_datapath_list(phase='test')
print(f'The length of testing set: {len(test_list)}')

## 2.3 Augumentation
Nhận thấy chúng ta có khá ít dữ liệu, ...

In [ ]:
import albumentations as A
from albumentations import Compose
from albumentations.pytorch import ToTensorV2
import cv2

transform = {
    'train': Compose([
        A.Rotate(p=0.1, limit=(-85, 80)),
        A.RandomShadow(
            num_shadows_lower=2, 
            num_shadows_upper=3, 
            shadow_dimension=3, 
            shadow_roi=(0, 0.7, 0.4, 0.8), 
            p=0.4
        ),
        A.ShiftScaleRotate(
            shift_limit=0.055, 
            scale_limit=0.065, 
            rotate_limit=35, 
            p=0.6
        ),
        A.RandomFog(
            fog_coef_lower=0.2, 
            fog_coef_upper=0.2, 
            alpha_coef=0.2, 
            p=0.3
        ),
        A.RGBShift(
            r_shift_limit=25, 
            g_shift_limit=15, 
            b_shift_limit=15, 
            p=0.3
        ),
        A.RandomBrightnessContrast(p=0.3),
        A.GaussNoise(
            var_limit=(50, 70),  
            always_apply=False, 
            p=0.3
        ),
        A.Resize(height=Config.img_size, width=Config.img_size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]),
    'val': Compose([
        A.Resize(Config.img_size, Config.img_size),
        A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),            
        ToTensorV2()
    ]),
    'test': Compose([
        A.Resize(Config.img_size, Config.img_size),
        A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
}

## 2.4 Create custom Dataset

In [ ]:
class PlantDataset(Dataset):
    """
    Class to create a Dataset
    
    Attributes
    ----------
    df_train : DataFrame
        DataFrame containing the image labels.
    file_list : list
        A list containing the paths to the images
    transform : object
        Instance of the preprocessing class (ImageTransform)
    phase : 'train' or 'val' or 'test'
        Specify whether to use train, validation, or test
    """
    def __init__(self, df_train, file_list, transform=None, phase='train'):
        self.df_train = df_train
        self.df_labels_idx = df_labels_idx
        self.file_list = file_list
        self.transform = transform[phase]
        self.phase = phase
        
    def __len__(self):
        """
        Returns the number of images.
        """
        return len(self.file_list)
    
    def __getitem__(self, index):
        """
        Get data in Tensor format and labels of preprocessed images.
        """
        
        # Load the index number image.
        img_path = self.file_list[index]
        img = Image.open(img_path)
        
        # Preprocessing images
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_transformed = self.transform(image=img)
        
        # image name
        image_name = img_path[-20:]
        
        # Extract the labels
        if self.phase in ["train", "val"]:
            label = df_train.loc[df_train["image"]==image_name]["encoded_label"].values[0]
        elif self.phase in ["test"]:
            label = -1
        
        return img_transformed, label, image_name

In [ ]:
train_dataset = PlantDataset(df_train, train_list, transform=transform, phase='train')
val_dataset = PlantDataset(df_train, val_list, transform=transform, phase='val')
test_dataset = PlantDataset(df_train, test_list, transform=transform, phase='test')

index = 0

print("【train dataset】")
print(f"img num : {train_dataset.__len__()}")
# print(f"img : {train_dataset.__getitem__(index)[0].size()}")
print(f"label : {train_dataset.__getitem__(index)[1]}")
print(f"image name : {train_dataset.__getitem__(index)[2]}")

print("\n【validation dataset】")
print(f"img num : {val_dataset.__len__()}")
# print(f"img : {val_dataset.__getitem__(index)[0].size()}")
print(f"label : {val_dataset.__getitem__(index)[1]}")
print(f"image name : {val_dataset.__getitem__(index)[2]}")

print("\n【test dataset】")
print(f"img num : {test_dataset.__len__()}")
# print(f"img : {test_dataset.__getitem__(index)[0].size()}")
print(f"label : {test_dataset.__getitem__(index)[1]}")
print(f"image name : {test_dataset.__getitem__(index)[2]}")

## 2.5 Create Dataloader

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=Config.batch_size, num_workers=2,shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=Config.batch_size, num_workers=2, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=Config.batch_size, num_workers=2, shuffle=False)

# to Dictionary
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader, "test": test_dataloader}

In [ ]:
for i, image_data in enumerate(train_dataloader):
    break

In [ ]:
plt.figure(figsize=(20, 20))

im = make_grid(image_data[0]['image'], nrow=8)
plt.imshow(np.transpose(im.numpy(), (1, 2, 0)))

# 3. Define model

## 3.1 Load model if exists 

In [ ]:
# # Load the Model back from file
# Pkl_Filename = '../input/pickle-test/Resmodel50_trained_1fc.pkl'
# with open(Pkl_Filename, 'rb') as file:  
#     model = pickle.load(file)

## 3.2 Define new model

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
model_config = {
    "name": "2 FCs, 0.0001 Lr, 30 Epochs",
    "classifier": torch.nn.Sequential(
                  torch.nn.Linear(2048, 1024),
                  torch.nn.Linear(1024, 12)),
    "lr": 0.001,
    "epoch": 20
}

In [ ]:
use_pretrained = True
pretrained_model = models.resnet152(pretrained=use_pretrained)

In [ ]:
name, classifier, lr, epoch = model_config.values()

pretrained_model.fc = classifier

# print(f'Model name: {name}')
# print(pretrained_model)

## Learning rate finder
https://towardsdatascience.com/the-learning-rate-finder-6618dfcb2025

In [ ]:
def lr_finder(model, min_lr, max_lr, dataset_lenght=train_dataset.__len__(), \
              batch_size=Config.batch_size, criterion=Config.criterion):
    iter_lrs = [min_lr]
    iter_losses = []
    
    factor = np.exp(np.log(max_lr / min_lr) / (dataset_lenght / batch_size))
    
    # Train model with 1 epoch
    model.to(Config.device)
    for i, data in tqdm(enumerate(dataloaders_dict['train']), total=len(dataloaders_dict['train'])):
        
        optimizer = Adam(model.parameters(), lr=min_lr)
        
        # set inputs, labels based on dataloader's batch data
        inputs = data[0]['image']
        labels = data[1]
        inputs = inputs.to(Config.device)
        labels = labels.to(Config.device)

        #zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # backward + optimize only if in training phase
        loss.backward()
        optimizer.step()
                
        # Update and append next iteration learning rate
        iter_lrs.append(min_lr)
        min_lr = min_lr * factor
        
        # Append this iteration loss
        iter_losses.append(np.log(loss.cpu().data.numpy().tolist()))
        
    iter_lrs.pop()
    
    # Plot loss vs log-scaled learning rate
    plt.figure(figsize=(10, 7))
    plot = sns.lineplot(iter_lrs, iter_losses)
    plot.set(xscale="log", 
             xlabel="Learning Rate (log-scale)", 
             ylabel="Training Loss",
             title="Optimal learning rate is slightly below minimum")

In [ ]:
# test_model = copy.deepcopy(pretrained_model)
# lr_finder(test_model, Config.min_lr, Config.max_lr)

Dựa vào biểu đồ, chúng ta có thể chọn learning rate của Adam = 10^-3. 

In [ ]:
# Config.lr = 10**-3
# optimizer = Adam(pretrained_model.parameters(), lr=Config.lr)
optimizer = Adam(pretrained_model.parameters(), lr=1e-3)

# 4. Huấn luyện và đánh giá model

### 4.1 Các biểu đồ của mô hình
> 1. Loss
> 2. Accuracy
> 3. F1

In [ ]:
def plot_result(train_losses, train_accuracy, train_f1, val_losses, val_accuracy, val_f1):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 7))
    ax1.plot(train_losses, label='Train')
    ax1.plot(val_losses, label='Validation')
    ax1.set_title('Loss')
    ax1.legend()

    ax2.plot(train_accuracy, label='Train')
    ax2.plot(val_accuracy, label='Validation')
    ax2.set_title('Accuracy')
    ax2.legend()

    ax3.plot(train_f1, label='Train')
    ax3.plot(val_f1, label='Validation')
    ax2.set_title('F1 Score')
    ax3.legend()

### 4.2 Train model

In [ ]:
def append_list(list, appended):
    for el in appended:
        list.append(el)
    return list

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=Config.epochs):
    
    train_losses = []
    train_accuracy = []
    train_f1 = []

    val_losses = []
    val_accuracy = []
    val_f1 = []
    
    print(f"Devices to be used : {Config.device}")
    model.to(Config.device)
    torch.backends.cudnn.benchmark = True
    
    start_time = time.time()
        
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            epoch_targets = []
            epoch_predictions = []

            # Iterate over data.
            for i, data in tqdm(enumerate(dataloaders_dict[phase]), total=len(dataloaders_dict[phase])):
#                 inputs = np.transpose(data[0]['image'], (0, 3, 1, 2))
                inputs = data[0]['image']
                labels = data[1]
                inputs = inputs.to(Config.device)
                labels = labels.to(Config.device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                np_preds = preds.cpu().data.numpy()
                np_labels = labels.cpu().data.numpy()
                append_list(epoch_predictions, np_preds)
                append_list(epoch_targets, np_labels)
                
                batch_f1 = f1_score(preds.cpu().data.numpy(), labels.cpu().data.numpy(), average='weighted')
                
#                 if i % 50 == 0 and i != 0:
#                 print(f'Batch: {i}  |  Loss: {loss.item():.4f}   |   F1-score: {batch_f1:.4f}%')         

            epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders_dict[phase].dataset)
            
            epoch_f1 = f1_score(epoch_predictions, epoch_targets, average='weighted')
            
            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accuracy.append(epoch_acc)
                train_f1.append(epoch_f1)
            else:
                val_losses.append(epoch_loss)
                val_accuracy.append(epoch_acc)
                val_f1.append(epoch_f1)
    
            print('{} Loss: {:.4f} Acc: {:.4f} F1_score: {:.4f}'.format('----> ' + phase.capitalize(), epoch_loss, epoch_acc, epoch_f1))
            
    print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed
    
    plot_result(train_losses, train_accuracy, train_f1, val_losses, val_accuracy, val_f1)
    
    return model

In [ ]:
trained_model = train_model(pretrained_model, Config.criterion, optimizer)

### 4.2 Save model

In [ ]:
def save_model(model, filename):
    Pkl_Filename = name + ".pkl"

    with open(Pkl_Filename, 'wb') as file:
        pickle.dump(model, file)
        
save_model(trained_model, name)

# 5. Make prediction

## 5.2 Create pridictor 

In [ ]:
class PlantPredictor():
    """
    Class for predicting labels from output results
    
    Attributes
    ----------
    df_labels_idx: DataFrame
        DataFrame that associates INDEX with a label name
    """
    
    def __init__(self, model, df_labels_idx, dataloaders_dict):
        self.model = model
        self.df_labels_idx = df_labels_idx
        self.dataloaders_dict = dataloaders_dict
        self.df_submit = pd.DataFrame()
        
    
    def __predict_max(self, out):
        """
        Get the label name with the highest probability.
        
        Parameters
        ----------
        predicted_label_name: str
            Name of the label with the highest prediction probability
        """
        maxid = np.argmax(out.detach().numpy(), axis=1)
        df_predicted_label_name = self.df_labels_idx.iloc[maxid]
        
        return df_predicted_label_name
    
    def inference(self):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        df_pred_list = []
        for i, data in enumerate(self.dataloaders_dict['test']):
            image_name = data[2]
            self.model.to(device)
            inputs = data[0]['image']
            inputs = inputs.to(device)
            out = self.model(inputs)
            device = torch.device("cpu")
            out = out.to(device)
            df_pred = self.__predict_max(out).reset_index(drop=True)
            df_pred["image"] = image_name
            df_pred_list.append(df_pred)
            
        self.df_submit = pd.concat(df_pred_list, axis=0)
        self.df_submit = self.df_submit[["image", "labels"]].reset_index(drop=True)

### 5.3 Make submission

In [ ]:
predictor = PlantPredictor(trained_model, df_labels_idx, dataloaders_dict)
predictor.inference()
df_submit = predictor.df_submit.copy()

In [ ]:
df_submit.to_csv('submission.csv', index=False)
df_submit